In [1]:
using LinearAlgebra
using Test

In [2]:
struct GivensMatrix{T} <: AbstractArray{T, 2}
	c::T
	s::T
	i::Int
	j::Int
	n::Int
end

In [3]:
Base.size(g::GivensMatrix) = (g.n, g.n)
# Base.size(g::GivensMatrix, i::Int) = i == 1 || i == 2 ? g.n : 1

In [4]:
function givens(A, i, j)
	x, y = A[i, 1], A[j, 1]
	norm = sqrt(x^2 + y^2)
	c = x/norm
	s = y/norm
	return GivensMatrix(c, s, i, j, size(A, 1))
end

givens (generic function with 1 method)

In [5]:
function left_mul!(A::AbstractMatrix, givens::GivensMatrix)
	for col in 1:3 # size(A, 2) 
		vi, vj = A[givens.i, col], A[givens.j, col]
		A[givens.i, col] = vi * givens.c + vj * givens.s
		A[givens.j, col] = -vi * givens.s + vj * givens.c
		# println(A)
		# println(1)
	end
	return A
end

left_mul! (generic function with 1 method)

In [6]:
function right_mul!(A::AbstractMatrix, givens::GivensMatrix)
	for row in 1:size(A, 1)   # size(A, 1)
		vi, vj = A[row, givens.i], A[row, givens.j]
		A[row, givens.i] = vi * givens.c + vj * givens.s
		A[row, givens.j] = -vi * givens.s + vj * givens.c
	end
	return A
end

right_mul! (generic function with 1 method)

In [7]:
function left_mul!_2(A::AbstractMatrix, givens::GivensMatrix)
	for col in 1:2
		#println(size(A, 1))
		vi, vj = A[givens.i, col], A[givens.j, col]
		A[givens.i, col] = vi * givens.c + vj * givens.s
		A[givens.j, col] = -vi * givens.s + vj * givens.c
		# println(A)
		# println(1)
	end
	return A
end

left_mul!_2 (generic function with 1 method)

In [8]:
function right_mul!_2(A::AbstractMatrix, givens::GivensMatrix)
	for row in 1:size(A, 1)
		# println(size(A, 1))
		vi, vj = A[row, givens.i], A[row, givens.j]
		A[row, givens.i] = vi * givens.c + vj * givens.s
		A[row, givens.j] = -vi * givens.s + vj * givens.c
	end
	return A
end

right_mul!_2 (generic function with 1 method)

In [9]:
function givens_qr!(Q::AbstractMatrix, A::AbstractMatrix)
	m, n = size(A)
	if m == 1
		return Q, A
	elseif m == 2
		k=2
		g = givens(A, k-1, k)
		left_mul!_2(A, g)
		right_mul!_2(Q, g)
		givens_qr!(view(Q, :, 2:m), view(A, 2:m, 2:n))
		return Q, A
	else
		for k =  2:-1:2
			g = givens(A, k-1, k)
			left_mul!(A, g)
			right_mul!(Q, g)
		end
		givens_qr!(view(Q, :, 2:m), view(A, 2:m, 2:n))
		return Q, A
	end
end

givens_qr! (generic function with 1 method)

In [10]:
n=6
A = [1. 1. 0. 0. 0. 0.; 1. 1. 1. 0. 0. 0.; 0. 1. 1. 1. 0. 0. ; 0. 0. 1. 1. 1. 0. ; 0. 0. 0. 1. 1. 1. ;0. 0. 0. 0. 1. 1.  ]
R = copy(A)
Q,R = givens_qr!(Matrix{Float64}(I, n, n), R)
R

6×6 Matrix{Float64}:
 1.41421  1.41421  0.707107     0.0       0.0       0.0
 0.0      1.0      1.0          1.0       0.0       0.0
 0.0      0.0      1.22474      0.816497  0.816497  0.0
 0.0      0.0      1.11022e-16  1.1547    1.1547    0.866025
 0.0      0.0      0.0          0.0       1.0       1.0
 0.0      0.0      0.0          0.0       0.0       0.5